In [ ]:
import pandas as pd
import numpy as np

# Hierarchical Indexing

### Multiindex

If you set an index to more than one columnn you are creating multi index or Hieararchical index. This makes asking questions based on indexes a lot more easier, and also opens the possibility of working with multidimensional data. 

We'll use the example sourced from [here](https://chrisalbon.com/python/pandas_hierarchical_data.html). 

In [ ]:
# Create dataframe
raw_data = {'regiment': ['Nighthawks', 'Nighthawks', 'Nighthawks', 'Nighthawks', 'Dragoons', 'Dragoons', 'Dragoons', 'Dragoons', 'Scouts', 'Scouts', 'Scouts', 'Scouts'], 
        'company': ['1st', '1st', '2nd', '2nd', '1st', '1st', '2nd', '2nd','1st', '1st', '2nd', '2nd'], 
        'name': ['Miller', 'Jacobson', 'Ali', 'Milner', 'Cooze', 'Jacon', 'Ryaner', 'Sone', 'Sloan', 'Piger', 'Riani', 'Ali'], 
        'preTestScore': [4, 24, 31, 2, 3, 4, 24, 31, 2, 3, 2, 3],
        'postTestScore': [25, 94, 57, 62, 70, 25, 94, 57, 62, 70, 62, 70]}
df = pd.DataFrame(raw_data, columns = ['regiment', 'company', 'name', 'preTestScore', 'postTestScore'])
df

In [ ]:
df_1_ind = df.set_index('regiment')
df_1_ind

In [ ]:
df_1_ind.mean(level = 'regiment')

* How about you want to get the mean scores, based on the company but not the regiment? 

In [ ]:
# Set the hierarchical index to be by regiment, and then by company
df_2_ind = df.set_index(['regiment', 'company'])
df_2_ind

<div class="alert alert-block alert-info">
<p>
Having multiple indexes will give you an easy way to model more than two dimensional data with DataFrames. Remember DataFraemes are by default a two dimensional data structures. 
</p>
<p>
For the above example, you can imagine each regiment is a two-dimensional array giving details about the company, names and the scores, and they are stacked one below the other. 
</p>
</div>

In [ ]:
df_2_ind.mean(level='company')

In [ ]:
df_2_ind.mean(level='regiment')

In [ ]:
df_2_ind.mean(level=['regiment','company'])

# Pandas Aggregation


We have already seen some simple aggregations on Pandas **`Series`** and **`DataFrame`** objects.

Let us review a few aggregation functions that will help us in understanding the **Grouping**. 

In [ ]:
# We'll be using our college scorecard dataset in this tutorial.
college_scorecard = pd.read_csv(
    './data/college-scorecard-data-scrubbed.csv', 
    encoding='latin-1')

In [ ]:
sat_averages = college_scorecard['sat_average']
print('Mean: {}'.format(sat_averages.mean()))
print('Max: {}'.format(sat_averages.max()))
print('Min: {}'.format(sat_averages.min()))
print('Median: {}'.format(sat_averages.median()))

<div class="alert alert-block alert-info">
<p>
Remember, that a series actually holds its values in a nested NumPy array (ndarray) object. Pandas simply has to apply these aggregations functions to that nested array.
</p>
</div>

Here is the list of available `Series` and `DataFrame` aggregation methods from your textbook.

| Aggregation Function      | Description    |      
|---------------|---------------------|
|count()        |Total number of items (not including NaN)|
|first(), last()|First and last item  |
|mean(), median()  |Mean and median   |
|min(), max()   |Minimum and Maximum  |
|std(), var()   |Standard deviation & variance |
|mad()          |Mean absolute deviation |
|prod()         |Product of all items         |
|sum()          |Sum of all items           |

### The `describe()` method
The `describe()` method is available on both **`Series`** and **`DataFrame`** objects and outputs a variety of aggregations that are very useful in getting the general "sense" of a dataset.

Take a look at the output for our **`sat_average`** series and **`college_scorecard`** dataframe.

In [ ]:
sat_averages.describe()

In [ ]:
college_scorecard.describe()

#### Tweaking `describe()` behavior with `include` and `exclude` parameters.
When used on a **`DataFrame`** object, the default behavior of the **`describe()`** method is to provide statistics on numeric columns only.

Let's take a look at the **`dtypes`** attribute on our college_scorecard dataframe to see what columns this does/doesn't include.

In [ ]:
college_scorecard.dtypes

<div class="alert alert-block alert-info">
<p>
The `dtype` attribute of `DataFrame` objects returns information on the datatype of each nested series/column.
</p>
</div>

See all the places where it lists the datatype of a column as 'object'? These columns won't be reported on with **`describe()`** when using the default parameters.

We can change this using either the **`include`** or the **`exclude`** parameters:

In [ ]:
# Include the object datatype columns
college_scorecard.describe(include=[np.object])

In [ ]:
# Exclude the numeric datatypes
college_scorecard.describe(exclude=[np.number])

There are two things here to notice:
1. The type of statistics returned changed when operating on **`object`** column types.
2. I used NumPy datatypes in the specification of what to include and exclude.

**The Statistics**  
Object(esp. string based) columns cannot be summarized reasonably with many of numeric aggregations so Pandas gives an alternative set of aggregations which make more sense for this type of data.

**NumPy Datatypes**  
Remember that the values of each `Series` inside of a `DataFrame` are stored in a NumPy array. Therefore the elements in that NumPy array are described by NumPy datatypes.

That is why we specify NumPy datatypes here to specifically include/exclude them for Pandas `describe` method.

This is just another example of the tight integration between the two libraries.

In [ ]:
# Finally, you can specify **`include='all'`** to force Pandas
# to evaluate all columns.  It will inject NaN where
# a calculation cannot be done.
college_scorecard.describe(include='all')

# Pandas Grouping

In this case we will look at the sample dataset of the flight schedules data that is available on Kaggle [here](https://www.kaggle.com/usdot/flight-delays)

This is only a sample of the original data. You will use the original data in your Group (no pun intended) Project!

In [ ]:
flights = pd.read_csv('./data/flight_sample.csv')
flights.head()

## The `groupby()` Method

So far, all the calculations that we've done on **`DataFrame`** objects have looked at the values of columns as a whole.

The `groupby()` method allows you to move into deeper forms analysis by splitting up the rows of a dataset into groups by the values in specified row(s). You can think of this in some ways as putting rows into buckets for evaluation.

### Specifying how to Split your Dataset into Groups
Of course, before we can perform evaluations on groups, we have to create them from an existing dataframe. 

Let's explore how **`groupby()`** provides a variety of ways to split up your datasets. We'll explore some of these here, starting with the most simple.

#### Single Column Grouping

In [ ]:
flights_by_airline = flights.groupby(['AIRLINE'])
flights_by_airline.groups

The **`groupby()`** method returns an type called **`DataFrameGroupBy`**. We will explore it in more depth shortly, but for now just know that it has an attribute called **`groups`** which provides a *`dict`* object with the **labels** of each group and the **corresponding index values** in the original dataframe that belong to that group.

If you look above, you can see there is a group labelled 'AA' will index values [2,   19,   43,   55,   59,   64,   71,   74,   82,   92, ...].

You can think of this as a record of all the groups that we will perform calculations on later.

#### Multi Column Grouping

You can specify multiple columns if you wish to split your data up in multiple levels:

In [ ]:
flights_by_airline_month = flights.groupby(['AIRLINE', 'MONTH'])
flights_by_airline_month.groups

### Aggregations after GroupBy

For example, let us say you want to find out the average distance traveled by each airline, you can do that using the following aggregeate function

In [ ]:
flights.head()

In [ ]:
flights_by_airline = flights.groupby(['AIRLINE'])

In [ ]:
flights_by_airline[['DISTANCE']].mean()

## Activity

##### Aggregation operations

1. Describe only numerical values of flights dataset. 
2. Describe only the non-numerical values of the flights dataset. 
3. Describe all the columns of the flights dataset. 


##### GroupBY

1. Extract only the flight details of the American Airlines (AA) 
2. What is the median monthly DISTANCE, TAXI_IN times and TAXI_OUT times? 
3. How about above summary statistics for United Airlines (UA)? 
4. Instead of doing this for each airline, what can you do so that you get all per airline per month summary statistics? 